# ResNet18 Satellite Feature Extraction (Optional)

This notebook processes 6-channel satellite imagery tiles through a **pre-trained** ResNet model and extracts features from the specified layer(s). This step is optional and can be performed to combine satellite features with geospatial data when executing `05_resnet18_fine_tuning.ipynb`


## File System Structure

## Input

The input satellite tiles (for each DHS location) are located in `Satellite_Tiles` within the hierarchy below. 
<pre style="font-family: monospace;">
./GIS-Image-Stack-Processing
    /AOI/
        PK/
            Image_Tiles/
                    :
            Satellite_Tiles/
                PK_1_C-1_30m.tif
                PK_2_C-2_30m.tif
                    :
                PK_560_C-580_30m.tif
                
            Satellite_Features/
                PK_sat_features_prithvi_L6_L8.npz (generated using 04_prithvi_sat_feature_extraction.ipynb)
                PK_sat_features_resnet_layer4.npz (geneerted using 04_resnet_sat_feature_extraction.ipynb)
</pre>


## Required Configurations

The following configurations are required for each execution of this notebook: the two-letter country code. Other model and feature extraction configurations are available in the Configuration section.
<pre style="font-family: monospace;">
<span style="color: blue;">country_code  = 'PK'</span>      # Set the country code to one of the available AOIs in the list below

Available AOIs: AM (Armenia)
                MA (Morocco)
                MB (Moldova)
                ML (Mali)
                MR (Mauritania)
                NI (Niger)
                PK (Pakistan)
                SN (Senegal)
                TD (Chad)
                
</pre>



In [1]:
#-------------------------------------------------
# REQUIRED CONFIGURATIONS HERE
#-------------------------------------------------
country_code  = 'ML'     # Set the country code
#-------------------------------------------------

In [2]:
import os
import sys
import re
import rasterio
import random
import numpy as np
import warnings
import json
from enum import Enum
from collections import Counter

import torch
import torch.nn as nn
import torchvision.transforms as T
from functools import partial

from torchvision.models import resnet18
from torchvision.models.resnet import ResNet18_Weights
from torchinfo import summary

from torch.utils.data import Dataset, DataLoader
from dataclasses import dataclass, field

In [3]:
# Set default num_workers
num_workers = 0

# Detect the OS name
os_name = os.popen('uname').read().strip()

# Check if the OS is Linux
if os_name == "Linux":
    
    print("Running on Linux. Setting num_workers to 64.")
    num_workers = 64
  
    print("Setting OS environment paths...")

    # Set CUDA_HOME to the conda environment prefix
    os.environ['CUDA_HOME'] = os.getenv('CONDA_PREFIX')

    # Update PATH to include the CUDA bin directory
    os.environ['PATH'] = os.path.join(os.getenv('CUDA_HOME'), 'bin') + ':' + os.getenv('PATH')

    # Update LD_LIBRARY_PATH to include the CUDA lib64 directory, handling the case where it's None
    ld_library_path = os.getenv('LD_LIBRARY_PATH')
    if ld_library_path is None:
        os.environ['LD_LIBRARY_PATH'] = os.path.join(os.getenv('CUDA_HOME'), 'lib64')
    else:
        os.environ['LD_LIBRARY_PATH'] = os.path.join(os.getenv('CUDA_HOME'), 'lib64') + ':' + ld_library_path

    # Set the environment variable for PyTorch CUDA memory allocation
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [4]:
sys.path.append('./GIS-Image-Stack-Processing')  # Adjust path if `gist_utils` is moved

cache_dir = 'project_utils/__pycache__'
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
    
from project_utils.aoi_configurations import *
from project_utils.resnet_utils import *
from project_utils.satellite_dataset_utils import HLSFlexibleBandSatDataset, custom_transforms

os.environ['PATH'] += os.pathsep + '/usr/local/bin/chromedriver'

#----------------------------------------------------------------------------------------
# *** IMPORTANT: SYSTEM PATH TO SET ***
#----------------------------------------------------------------------------------------
# The following path is required, as it contains GDAL binaries used for several 
# pre-processing functions. The pathname corresponds to the Conda virtual environment 
# created for this project (e.g., "py39-pt").
#
# Note: GDAL was adopted as a benchmark to compare the original GIS data produced by 
# another team. However, similar functionality could be implemented using the Rasterio 
# Python package. If Rasterio is used, it would eliminate the need for GDAL binaries 
# and this system path specification.
#----------------------------------------------------------------------------------------

# Adding path to gdal commands for local system
os.environ['PATH'] += ':/Users/billk/miniforge3/envs/py39-pt/bin/' 

## Confirm Number of Bands in HLS Satellite Data

In [5]:
#run_gdalinfo(f"./GIS-Image-Stack-Processing/AOI/{country_code}/Satellite_Tiles/{country_code}_1_C-1_30m.tif")

## System Configuration

In [6]:
def system_config(SEED_VALUE=42):
    """
    Configures the system environment for PyTorch-based operations.

    Args:
        SEED_VALUE (int): Seed value for random number generation. 
        package_list (str): String containing a list of additional packages to install  
        for Google Colab or Kaggle. 

    Returns:
        tuple: A tuple containing the device name as a string and a boolean indicating GPU availability.
    """

    random.seed(SEED_VALUE)
    np.random.seed(SEED_VALUE)
    torch.manual_seed(SEED_VALUE)

    def is_running_in_colab():
        return 'COLAB_GPU' in os.environ
        
    def is_running_in_kaggle():
        return 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

    #--------------------------------
    # Check for availability of GPUs. 
    #--------------------------------
    if torch.cuda.is_available():
        print('Using CUDA GPU')
        
        # Set the device to the first CUDA device.
        DEVICE = torch.device('cuda')
        print("Device: ", DEVICE)
        GPU_AVAILABLE = True

        torch.cuda.manual_seed(SEED_VALUE)
        torch.cuda.manual_seed_all(SEED_VALUE)

        # Performance and deterministic behavior.
        torch.backends.cudnn.enabled = True       # Provides highly optimized primitives for DL operations.
        torch.backends.cudnn.deterministic = False 
        torch.backends.cudnn.benchmark = False    # Setting to True can cause non-deterministic behavior.
        
    else:
        
        print('Using CPU')
        DEVICE = torch.device('cpu')
        print("Device: ", DEVICE)
        GPU_AVAILABLE = False
        
        if is_running_in_colab() or is_running_in_kaggle():
            print('Installing required packages...')
            !pip install {package_list}
            print('Note: Change runtime type to GPU for better performance.')
        
        torch.use_deterministic_algorithms(True)

    return str(DEVICE), GPU_AVAILABLE

In [7]:
DEVICE, GPU_AVAILABLE = system_config()

if DEVICE == 'cuda':
    torch.cuda.empty_cache()
    !nvidia-smi

Using CPU
Device:  cpu


## Model and Data Configuration

In [8]:
class ModelMode(Enum):
    PRE_TRAINED = "Pre_Trained"  # Only valid option is: PRE_TRAINED
    
# Dataset configuration parameters
@dataclass(frozen=True)
class DatasetConfig:
    COUNTRY_CODE:     str  
    IMG_HEIGHT:       int = 224
    IMG_WIDTH:        int = 224
    GIS_ROOT:         str = './GIS-Image-Stack-Processing'
    AOI_ROOT:         str = './GIS-Image-Stack-Processing/AOI/'
    PRT_ROOT:         str = './GIS-Image-Stack-Processing/AOI/Partitions'

@dataclass(frozen=True)
class FeatureConfig:
    FEATURE_LAYER:   str = 'layer4'
    BLOCK_INDEX:     int = 1
    SUB_LAYER_PART:  str = 'conv2'
    RELU:            bool = True        # Set to True to extract featuers from last (ReLU) in layer.
                                        # Ignores BLOCK_INDEX and SUB_LAYER_PART
@dataclass(frozen=True)
class TrainingConfig:
    BATCH_SIZE:       int   = 8
    NUM_WORKERS:      int   = num_workers
    MODEL_MODE:       ModelMode = ModelMode.PRE_TRAINED  
    LOG_DIR:          str   = "./ResNet18_LOGS_DATA"
    CASE_STRING:      str   = "HLS"                    # Optional: Additional case string 
    
    # ImageNet values
    MEAN_STD: dict = field(default_factory=lambda: {
        'R': (0.485, 0.229),
        'G': (0.456, 0.224),
        'B': (0.406, 0.225)
    }) 

In [9]:
dataset_config = DatasetConfig(COUNTRY_CODE=country_code)
                               
train_config = TrainingConfig()

feature_config = FeatureConfig()

if feature_config.RELU:
    extraction_layer = feature_config.FEATURE_LAYER
else:
    extraction_layer = feature_config.FEATURE_LAYER + \
                        '.' + str(feature_config.BLOCK_INDEX) + \
                        '.' + feature_config.SUB_LAYER_PART
    
#------------------------------------------
# *** Satellite featue file name ***
#------------------------------------------
sat_feature_file  = f'{country_code}_sat_features_resnet_{extraction_layer}.npz'

print('\n')
print("Extraction layer: ", extraction_layer)
print('\n')

aoi_target_json_path = os.path.join(dataset_config.GIS_ROOT, f'AOI/{country_code}/Targets/targets.json')

training_string = train_config.MODEL_MODE.value
print('Training string: ', training_string)



Extraction layer:  layer4


Training string:  Pre_Trained


## Load DHS Cluster Data and Target Values from AOI  `targets.json`

In [10]:
dhs_df, geospatial_df = process_aoi_target_json(aoi_target_json_path, country_code)

   cluster_id     lat     lon  fraction_dpt3_vaccinated  \
0           1  14.530 -11.324                     0.778   
1           2  14.789 -11.927                     0.231   
2           3  14.577 -11.844                     0.100   
3           4  15.105 -11.819                     0.167   
4           5  14.735 -11.114                     0.182   

   fraction_with_electricity  fraction_with_fresh_water  mean_wealth_index  \
0                      0.600                       1.00              0.750   
1                      0.680                       0.96              0.700   
2                      0.714                       1.00              0.643   
3                      0.421                       1.00              0.671   
4                      0.750                       1.00              0.625   

   fraction_with_radio  fraction_with_tv country_code  
0                0.040             0.160           ML  
1                0.040             0.160           ML  
2       

In [11]:
# Define mean and std from train_config for normalization
mean = [train_config.MEAN_STD['R'][0], train_config.MEAN_STD['G'][0], train_config.MEAN_STD['B'][0]]
std  = [train_config.MEAN_STD['R'][1], train_config.MEAN_STD['G'][1], train_config.MEAN_STD['B'][1]]

img_size = (dataset_config.IMG_HEIGHT, dataset_config.IMG_WIDTH)

# Define the transform
transform = lambda image: custom_transforms(image, mean=mean, std=std, img_size=img_size)

##  Create `dataset` and `data_loader`

In [12]:
aoi_partition   = os.path.join(dataset_config.PRT_ROOT, f'{country_code}', f'{country_code}_all.json')

# Define the transform with required arguments
transform = partial(custom_transforms, mean=mean, std=std, img_size=img_size)

# Used to access AOI data for data exploration (not related to model training)
print('\n')
aoi_dataset = HLSFlexibleBandSatDataset(root_dir=dataset_config.AOI_ROOT,
                                        partition_map_path=aoi_partition, 
                                        num_channels=3,
                                        transform=transform)
print('\n')
aoi_data_loader   = DataLoader(aoi_dataset,   
                               batch_size=train_config.BATCH_SIZE, 
                               num_workers=train_config.NUM_WORKERS,
                               persistent_workers=False,
                               shuffle=False)

print("Number of samples in the aoi data loader: ",   len(aoi_data_loader.dataset))



Processing AOI: ML, 322 clusters


Number of samples in the aoi data loader:  322


## Model Instantiation

In [13]:
pretrained_model, features_list = get_resnet_18(output_features=1, 
                                                extraction_layer=extraction_layer,
                                                fine_tune_layers=0)

print(summary(pretrained_model,
              input_size=(1, 3, dataset_config.IMG_HEIGHT, dataset_config.IMG_WIDTH),
              row_settings=["var_names"])) 


Setting hook for extraction layer: layer4
Extraction layer is a high-level layer: layer4
Attaching hook to the last block in layer4
Layer (type (var_name))                  Output Shape              Param #
ResNet (ResNet)                          [1, 1]                    --
├─Conv2d (conv1)                         [1, 64, 112, 112]         (9,408)
├─BatchNorm2d (bn1)                      [1, 64, 112, 112]         (128)
├─ReLU (relu)                            [1, 64, 112, 112]         --
├─MaxPool2d (maxpool)                    [1, 64, 56, 56]           --
├─Sequential (layer1)                    [1, 64, 56, 56]           --
│    └─BasicBlock (0)                    [1, 64, 56, 56]           --
│    │    └─Conv2d (conv1)               [1, 64, 56, 56]           (36,864)
│    │    └─BatchNorm2d (bn1)            [1, 64, 56, 56]           (128)
│    │    └─ReLU (relu)                  [1, 64, 56, 56]           --
│    │    └─Conv2d (conv2)               [1, 64, 56, 56]           (36,864)


## Extract (HLS Satellite) ResNet18 Features

In [14]:
pretrained_model = pretrained_model.float().to(DEVICE)
pretrained_model.eval() 

features_resnet, cluster_ids, target_values_list = extract_features(pretrained_model, 
                                                                    aoi_data_loader, 
                                                                    features_list,
                                                                    device=DEVICE)
print("Using Pre-Trained Model")

Using Pre-Trained Model


In [15]:
print(features_resnet.shape)
print(len(cluster_ids))
# print(cluster_ids)

(322, 25088)
322


## Save Features to Disk

In [16]:
satellite_features_folder = f'{dataset_config.AOI_ROOT}/{country_code}/Satellite_Features'

if not os.path.exists(satellite_features_folder):
    os.makedirs(satellite_features_folder)

feature_file = f'{satellite_features_folder}/{sat_feature_file}'

np.savez(feature_file,
         features=features_resnet,
         cluster_ids=cluster_ids)

print(f"Features saved to {feature_file}")

Features saved to ./GIS-Image-Stack-Processing/AOI//ML/Satellite_Features/ML_sat_features_resnet_layer4.npz
